# Making a CEX File from a Text File




## Configuring CITE libraries for almond kernel

First, we'll make a bintray repository with CITE libraries available to your almond kernel.

In [118]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [119]:
import $ivy.`edu.holycross.shot::ohco2:10.16.0`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:5.2.2`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`
import $ivy.`edu.holycross.shot::greek:2.3.3`


import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                              

import $ivy.$                              

import $ivy.$                                          

import $ivy.$                                  

import $ivy.$                                        

import $ivy.$                              

import $ivy.$                                


## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [120]:
// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.greek._

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source


import edu.holycross.shot.cite._

import edu.holycross.shot.ohco2._

import edu.holycross.shot.scm._

import edu.holycross.shot.citeobj._

import edu.holycross.shot.citerelation._

import edu.holycross.shot.dse._

import edu.holycross.shot.citebinaryimage._

import edu.holycross.shot.ohco2._

import edu.holycross.shot.greek._


import almond.display.UpdatableDisplay

import almond.interpreter.api.DisplayData.ContentType

import almond.interpreter.api.{DisplayData, OutputHandler}


import java.io.File

import java.io.PrintWriter


import scala.io.Source


## Useful Functions

Save a string:

In [121]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

defined function saveString

Pretty Print many things:

In [122]:
def showMe(v:Any):Unit = {
  v match {
    case _:StringHistogram => {
        for ( h <- v.asInstanceOf[StringHistogram].histogram ) {
            println(s"${h.count}\t${h.s}")
        }
    }
  	case _:Corpus => {
  		for ( n <- v.asInstanceOf[Corpus].nodes) {
  			println(s"${n.urn.passageComponent}\t\t${n.text}")
  		}	
  	}
    case _:Vector[Any] => println(s"""\n----\n${v.asInstanceOf[Vector[Any]].mkString("\n")}\n----\n""")
    case _:Iterable[Any] => println(s"""\n----\n${v.asInstanceOf[Iterable[Any]].mkString("\n")}\n----\n""")
    case _ => println(s"\n-----\n${v}\n----\n")
  }
}

defined function showMe

## Load a Template File

Make the CEX Header

In [123]:
val cexTop: String = """
#!cexversion
3.0

#!citelibrary
name#CEX library
urn#urn:cite2:cex:TEMPCOLL.TEMPVERSION:TEMP_ID
license#CC 3.0 NC-BY

#!ctscatalog
urn#citationScheme#groupName#workTitle#versionLabel#exemplarLabel#online#lang"""

val urnStr = "urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:"
val citationSch = "section/subsection"
val groupName = "Aristotle"
val workTitle = "Poetics"
val versionLabel = "W.H. Fyfe, trans. 1932"
val exemplarLabel = ""
val online = "true"
val lang = "eng"

val headerLine = Vector(
    urnStr,
    citationSch,
    groupName,
    workTitle,
    versionLabel,
    exemplarLabel,
    online,
    lang
).mkString("#")

val cexHeader = cexTop + "\n" + headerLine + "\n\n#!ctsdata\n"

cexTop: String = """
#!cexversion
3.0

#!citelibrary
name#CEX library
urn#urn:cite2:cex:TEMPCOLL.TEMPVERSION:TEMP_ID
license#CC 3.0 NC-BY

#!ctscatalog
urn#citationScheme#groupName#workTitle#versionLabel#exemplarLabel#online#lang"""
urnStr: String = "urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:"
citationSch: String = "section/subsection"
groupName: String = "Aristotle"
workTitle: String = "Poetics"
versionLabel: String = "W.H. Fyfe, trans. 1932"
exemplarLabel: String = ""
online: String = "true"
lang: String = "eng"
headerLine: String = "urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:#section/subsection#Aristotle#Poetics#W.H. Fyfe, trans. 1932##true#eng"
cexHeader: String = """
#!cexversion
3.0

#!citelibrary
name#CEX library
urn#urn:cite2:cex:TEMPCOLL.TEMPVERSION:TEMP_ID
license#CC 3.0 NC-BY

#!ctscatalog
urn#citationScheme#groupName#workTitle#versionLabel#exemplarLabel#online#lang
urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:#section/subsection#Aristotle#Poetics#W.H. Fyfe, trans. 1932##true#eng

#!c

Load it the (pre-processed) text file:

In [124]:
val filePath = s"pre_cex_eng.txt"
val lines: Vector[String] = {
    scala.io.Source.fromFile(filePath).mkString.split("\n").toVector.filter( _.size > 0 )
}

filePath: String = "pre_cex_eng.txt"
lines: Vector[String] = Vector(
  "head#Poetics ",
  "1.1# Let us here deal with Poetry, its essence and its several species, with the characteristic function of each species and the way in which plots must be constructed if the poem is to be a success; and also with the number and character of the constituent parts of a poem, and similarly with all other matters proper to this same inquiry; and let us, as nature directs, begin first with first principles.  ",
  "1.2# Epic poetry, then, and the poetry of tragic drama, and, moreover, comedy and dithyrambic poetry, and most flute-playing and harp-playing, these, speaking generally, may all be said to be \"representations of life.\" ",
  "1.3# But they differ one from another in three ways: either in using means generically different or in representing different objects or in representing objects not in the same way but in a different manner.  ",
  "1.4#For just as by the use both of color and form peo

We are looking for embedded Greek words. They will look like this: `<f>A)/GW</f>`. That is, an XML `<f>` element containing Greek in Beta-Code.

We'll start by defining a Regular Expression that will match these:

In [125]:
val fMatcher = "<f>([^<]+)</f>".r

fMatcher: Regex = <f>([^<]+)</f>

We make a Function to return the contents of thos `<f>` elements, for a string:

In [126]:
def getGroupsInMatch( s: String, rx: scala.util.matching.Regex, grp: Int ): Vector[String] = {
    val string = s
    val pattern = rx
    pattern.findAllIn(string).matchData.map( md => {
        md.group(grp)
    }).toVector
}



defined function getGroupsInMatch

We create an overloaded function to report `true` or `false`, based on whether a string matches our regex. (There is no point doing all this to a line that has no `<f>` element!)

In [127]:
def containsMatch( s: String, rx: scala.util.matching.Regex): Boolean = {
    rx.findAllIn(s).size > 0
}

def containsMatch( s: String, rs: String): Boolean = {
    s.contains(rs)
}

defined function containsMatch
defined function containsMatch

We make a little function to use the Greek library to convert beta-code to Unicode:

In [128]:
def betaToUnicode( bc: String ): String = {
    val gs = LiteraryGreekString(bc)
    gs.ucode
}

defined function betaToUnicode

We want a Map that has the `<f>` element as they key, and the unicode text (minus the tags) as the value:

In [129]:
def matchesToUcodeReplacementMap( v: Vector[String] ): Map[String, String] = {
    val map: Map[String, String] = v.distinct.map( b => {
        val beta: String = b
        val uni: String = betaToUnicode(b.toLowerCase)
        (s"<f>${b}</f>", uni)
    }).toMap
    map
}

defined function matchesToUcodeReplacementMap

This is the fancy, recursive way we go through a map of replacement-pairs, and replace them, while never using mutable data. 

> **Note** `str.replace(replVec.head._1.toCharArray, replVec.head._2.toCharArray)` is what we use here instead of `str.replaceAll(s, r)`. `.replaceAll` will treat the first parameter as a regular expression; the parenthese and other marks in Beta Code will throw errors. This is the workaround.

In [130]:

def recursiveReplace( str: String, replMap: Map[String, String]): String = {
    
    def doIt( str: String, replVec: Vector[(String, String)]): String = {
        if (replVec.size == 1) {
            val newStr: String = str.replace(replVec.head._1.toCharArray, replVec.head._2.toCharArray)
            newStr
        } else {
            val newStr: String = str.replace(replVec.head._1.toCharArray, replVec.head._2.toCharArray)
            doIt( newStr, replVec.tail)
        }
    }
    
    doIt( str, replMap.toVector )
}



defined function recursiveReplace

Let's test this complex setup!

In [131]:
val testStr: String = """3.6# Their name, they say, for suburb villages is <f>KW=MAI</f>—the Athenians call them "Demes"—and comedians are so called not from <f>KWMA/ZEIN</f>, "to revel," but because they were turned out of the towns and went strolling round the villages( <f>KW=MAI</f>). Their word for action, they add, is <f>DRA=N</f>, whereas the Athenian word is <f>PRA/TTEIN</f>. So much then for the differences, their number, and their nature. """

val matchVec: Vector[String] = getGroupsInMatch(testStr, fMatcher, 1)
val replaceMap: Map[String, String] = matchesToUcodeReplacementMap(matchVec)
val testRepl: String = recursiveReplace( testStr, replaceMap )
println(testRepl)

3.6# Their name, they say, for suburb villages is κῶμαι—the Athenians call them "Demes"—and comedians are so called not from κωμάζειν, "to revel," but because they were turned out of the towns and went strolling round the villages( κῶμαι). Their word for action, they add, is δρᾶν, whereas the Athenian word is πράττειν. So much then for the differences, their number, and their nature. 


testStr: String = "3.6# Their name, they say, for suburb villages is <f>KW=MAI</f>\u2014the Athenians call them \"Demes\"\u2014and comedians are so called not from <f>KWMA/ZEIN</f>, \"to revel,\" but because they were turned out of the towns and went strolling round the villages( <f>KW=MAI</f>). Their word for action, they add, is <f>DRA=N</f>, whereas the Athenian word is <f>PRA/TTEIN</f>. So much then for the differences, their number, and their nature. "
matchVec: Vector[String] = Vector(
  "KW=MAI",
  "KWMA/ZEIN",
  "KW=MAI",
  "DRA=N",
  "PRA/TTEIN"
)
replaceMap: Map[String, String] = Map(
  "<f>KW=MAI</f>" -> "\u03ba\u1ff6\u03bc\u03b1\u03b9",
  "<f>KWMA/ZEIN</f>" -> "\u03ba\u03c9\u03bc\u03ac\u03b6\u03b5\u03b9\u03bd",
  "<f>DRA=N</f>" -> "\u03b4\u03c1\u1fb6\u03bd",
  "<f>PRA/TTEIN</f>" -> "\u03c0\u03c1\u03ac\u03c4\u03c4\u03b5\u03b9\u03bd"
)
testRepl: String = "3.6# Their name, they say, for suburb villages is \u03ba\u1ff6\u03bc\u03b1\u03b9\u2014the Athenians call them \"Demes\"\u2

We want to map each line of text (the Vector `lines`), looking for Greek words. When we find some, we will turn Beta-code into Unicode using the library [`edu.holycross.shot.greek`](https://neelsmith.github.io/greek/).

In [132]:
val greekedLines: Vector[String] = {
    lines.map( l => {
        if ( containsMatch( l, fMatcher) ) {
           val matchVec: Vector[String] = getGroupsInMatch(l, fMatcher, 1)
           val replaceMap: Map[String, String] = matchesToUcodeReplacementMap(matchVec)
           recursiveReplace( l, replaceMap )
        } else l
    })
}

greekedLines: Vector[String] = Vector(
  "head#Poetics ",
  "1.1# Let us here deal with Poetry, its essence and its several species, with the characteristic function of each species and the way in which plots must be constructed if the poem is to be a success; and also with the number and character of the constituent parts of a poem, and similarly with all other matters proper to this same inquiry; and let us, as nature directs, begin first with first principles.  ",
  "1.2# Epic poetry, then, and the poetry of tragic drama, and, moreover, comedy and dithyrambic poetry, and most flute-playing and harp-playing, these, speaking generally, may all be said to be \"representations of life.\" ",
  "1.3# But they differ one from another in three ways: either in using means generically different or in representing different objects or in representing objects not in the same way but in a different manner.  ",
  "1.4#For just as by the use both of color and form people represent many objects, ma

In [133]:
greekedLines.filter( _.contains("##"))

res132: Vector[String] = Vector()

## Final Assembly

We can now map our new string vector to prepend the urn. We'll do a little space-normalization while we're at it:

In [139]:
val cexTextVec: Vector[String] = greekedLines.map( l => {
    s"${urnStr}${l.trim}".replaceAll(" +"," ")
                         .replaceAll("# ","#")
                         .replaceAll(""" " """, """ """")
})

cexTextVec: Vector[String] = Vector(
  "urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:head#Poetics",
  "urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:1.1#Let us here deal with Poetry, its essence and its several species, with the characteristic function of each species and the way in which plots must be constructed if the poem is to be a success; and also with the number and character of the constituent parts of a poem, and similarly with all other matters proper to this same inquiry; and let us, as nature directs, begin first with first principles.",
  "urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:1.2#Epic poetry, then, and the poetry of tragic drama, and, moreover, comedy and dithyrambic poetry, and most flute-playing and harp-playing, these, speaking generally, may all be said to be \"representations of life.\"",
  "urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:1.3#But they differ one from another in three ways: either in using means generically different or in representing different objects or in represe

And we make it One Big String:

In [140]:
val cexText: String = cexTextVec.mkString("\n")
val finalCex: String = cexHeader + cexText

cexText: String = """urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:head#Poetics
urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:1.1#Let us here deal with Poetry, its essence and its several species, with the characteristic function of each species and the way in which plots must be constructed if the poem is to be a success; and also with the number and character of the constituent parts of a poem, and similarly with all other matters proper to this same inquiry; and let us, as nature directs, begin first with first principles.
urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:1.2#Epic poetry, then, and the poetry of tragic drama, and, moreover, comedy and dithyrambic poetry, and most flute-playing and harp-playing, these, speaking generally, may all be said to be "representations of life."
urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:1.3#But they differ one from another in three ways: either in using means generically different or in representing different objects or in representing objects not in the same way bu

And we save it!

In [141]:
val fileName = "aristot_poetics_eng.cex"

fileName: String = "aristot_poetics_eng.cex"

In [142]:
saveString(finalCex, "cex/", fileName)

## Final Validation

We want to confirm that we can load this library!

In [143]:
val cexPath = "cex/aristot_poetics_eng.cex"
val lib = CiteLibrary(scala.io.Source.fromFile(cexPath).mkString)

Feb 12, 2020 12:23:18 AM wvlet.log.Logger log
INFO: Building text repo from cex ...
Feb 12, 2020 12:23:18 AM wvlet.log.Logger log
INFO: Building collection repo from cex ...
Feb 12, 2020 12:23:18 AM wvlet.log.Logger log
INFO: Building relations from cex ...
Feb 12, 2020 12:23:18 AM wvlet.log.Logger log
INFO: All library components built.


cexPath: String = "cex/aristot_poetics_eng.cex"
lib: CiteLibrary = CiteLibrary(
  "CEX library",
  Cite2Urn("urn:cite2:cex:TEMPCOLL.TEMPVERSION:TEMP_ID"),
  "CC 3.0 NC-BY",
  Vector(),
  Some(
    TextRepository(
      Corpus(
        Vector(
          CitableNode(
            CtsUrn("urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:head"),
            "Poetics"
          ),
          CitableNode(
            CtsUrn("urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:1.1"),
            "Let us here deal with Poetry, its essence and its several species, with the characteristic function of each species and the way in which plots must be constructed if the poem is to be a success; and also with the number and character of the constituent parts of a poem, and similarly with all other matters proper to this same inquiry; and let us, as nature directs, begin first with first principles."
          ),
          CitableNode(
            CtsUrn("urn:cts:greekLit:tlg0086.tlg034.fyfe_fu:1.2"),
            "Epic poet